In [ ]:
# Set your file path
file_path = "./Temp data/0.csv"

# Step 1: Extract Metadata (first two lines)
with open(file_path, "r", encoding="utf-8") as file:
    meta_lines = [next(file).strip() for _ in range(2)]

# Convert metadata to dictionary
meta_keys = meta_lines[0].split(",")
meta_values = meta_lines[1].split(",")
metadata = dict(zip(meta_keys, meta_values))

# Step 2: Load actual data (skip the blank line after metadata)
data_df = pd.read_csv(file_path, skiprows=3)

# Optional: Display results
print("Metadata:")
for k, v in metadata.items():
    print(f"{k}: {v}")

print("\nStructured Data Sample:")
print(data_df.head())

Metadata:
latitude: 25.764498
longitude: -80.39215
elevation: 5.0
utc_offset_seconds: -14400
timezone: America/New_York
timezone_abbreviation: GMT-4

Structured Data Sample:
               time  temperature_2m (°C)  relative_humidity_2m (%)  \
0  1999-03-07T00:00                 18.4                      71.0   
1  1999-03-07T01:00                 19.4                      65.0   
2  1999-03-07T02:00                 18.8                      68.0   
3  1999-03-07T03:00                 15.2                      88.0   
4  1999-03-07T04:00                 15.4                      87.0   

   dew_point_2m (°C)  precipitation (mm)  wind_speed_10m (km/h)  \
0               13.1                 0.0                    2.9   
1               12.7                 0.0                    2.3   
2               12.7                 0.0                    6.3   
3               13.2                 0.0                    7.2   
4               13.2                 0.0                    8.4   

  

In [20]:
data_df.shape

(228312, 7)

In [ ]:
import os
import pandas as pd

weather_dir = "./Temp data"  # folder containing your 62 CSV files
all_weather_annual = []

# Loop over every CSV file in the directory
for file in os.listdir(weather_dir):
    if file.endswith(".csv"):
        # Extract cluster id from the file name (assuming it's just the number)
        cluster_id = int(os.path.splitext(file)[0])
        file_path = os.path.join(weather_dir, file)

        # --- Step 1: Read the metadata from the first two lines ---
        with open(file_path, "r", encoding="utf-8") as f:
            meta_lines = [next(f).strip() for _ in range(2)]
        # Convert metadata lines into a dictionary
        meta_keys = [k.strip() for k in meta_lines[0].split(",")]
        meta_values = [v.strip() for v in meta_lines[1].split(",")]
        metadata = dict(zip(meta_keys, meta_values))

        # --- Step 2: Read the weather data (skip metadata lines and blank line) ---
        weather_df = pd.read_csv(file_path, skiprows=3, low_memory=False)

        # Convert the 'time' column to datetime and extract the year
        weather_df["time"] = pd.to_datetime(weather_df["time"], errors="coerce")
        weather_df["year"] = weather_df["time"].dt.year

        # --- Step 3: Aggregate the data by year (e.g., take the mean) ---
        weather_annual = weather_df.groupby("year").mean().reset_index()

        # --- Step 4: Attach the metadata values (if needed) ---
        for key in ["latitude", "longitude", "elevation"]:
            if key in metadata:
                weather_annual[key] = float(metadata[key])
            else:
                weather_annual[key] = None

        # Attach the cluster id
        weather_annual["cluster"] = cluster_id

        # Append to the list
        all_weather_annual.append(weather_annual)

# Combine all clusters into one master DataFrame
master_weather_df = pd.concat(all_weather_annual, ignore_index=True)

print("Master Aggregated Weather Data Sample:")
print(master_weather_df.head())

Master Aggregated Weather Data Sample:
   year                time  temperature_2m (°C)  dew_point_2m (°C)  \
0  1999 1999-07-02 11:30:00            15.743790          11.126963   
1  2000 2000-07-01 23:30:00            16.523634          12.222484   
2  2001 2001-07-02 11:30:00            15.837397          12.031689   
3  2002 2002-07-02 11:30:00            15.704144          11.153550   
4  2003 2003-07-02 11:30:00            16.461678          12.182397   

   relative_humidity_2m (%)  wind_speed_10m (km/h)  \
0                 76.019749               9.943231   
1                 77.271630               9.615073   
2                 79.333105               9.645776   
3                 76.475571               9.544726   
4                 77.650799               9.681530   

   soil_temperature_0_to_7cm (°C)  precipitation (mm)   latitude  longitude  \
0                       19.322911            0.022557  32.583477 -117.07674   
1                       20.152527            0.0227

In [18]:
master_weather_df.shape

(1674, 12)

In [ ]:
import pandas as pd

# Load the datacenter information
datacenter_df = pd.read_csv("top_clusters_with_midpoints.csv")

# Display the first few rows to inspect the structure
print("Datacenter Data (first 5 rows):")
print(datacenter_df.head())

# Optionally, check the columns and data types
print("\nDatacenter Data Info:")
print(datacenter_df.info())

Datacenter Data (first 5 rows):
    latitude  longitude                              name  \
0  25.791496 -80.379582        QTS · 11234 NW 20th Street   
1  25.793027 -80.381242                EdgeConneX · MIA01   
2  25.796127 -80.379946  Radius DC · 11300 NW 25th Street   
3  27.689285 -99.453613                 MDC Laredo - LDO1   
4  27.690588 -99.449913           MDC Data Centers · LDO1   

                  provider                                       full_address  \
0                      QTS               11234 NW 20th Street, Miami, FL, USA   
1               EdgeConnex               2132 NW 114th Avenue, Miami, FL, USA   
2                 RadiusDC         11300 NW 25th Street, Sweet Water, FL, USA   
3         MDC Data Centers           13619 Cabezut Dr, Laredo, TX, 78045, USA   
4  Media Networks Services  13619 Cabezut Drive, Unitec Industrial Park, L...   

   data_center_count  aggregated_power  aggregated_area  yearbuilt  \
0                  1              2.00      

In [ ]:
# Define the range of years based on your weather data
years_available = list(range(1999, 2026))  # 1999 to 2025 inclusive

# Create an empty list to store expanded rows
expanded_rows = []

# Loop over each datacenter record and add a row for each year
for _, row in datacenter_df.iterrows():
    for yr in years_available:
        new_row = row.copy()  # Copy the original datacenter row
        new_row["year"] = yr  # Add the current year for merging with weather data

        # Create an operational indicator: 1 if the current year is >= yearbuilt, else 0
        new_row["operational"] = int(yr >= row["yearbuilt"])

        expanded_rows.append(new_row)

# Create the expanded DataFrame
expanded_datacenter_df = pd.DataFrame(expanded_rows)

# Display the first few rows of the expanded DataFrame
print("\nExpanded Datacenter DataFrame (first 10 rows):")
print(expanded_datacenter_df.head(10))

# Optionally, check the new shape: should be original rows times number of years (n * 27 if 27 years)
print("\nOriginal DataFrame rows:", len(datacenter_df))
print("Expanded DataFrame rows:", len(expanded_datacenter_df))


Expanded Datacenter DataFrame (first 10 rows):
    latitude  longitude                        name provider  \
0  25.791496 -80.379582  QTS · 11234 NW 20th Street      QTS   
0  25.791496 -80.379582  QTS · 11234 NW 20th Street      QTS   
0  25.791496 -80.379582  QTS · 11234 NW 20th Street      QTS   
0  25.791496 -80.379582  QTS · 11234 NW 20th Street      QTS   
0  25.791496 -80.379582  QTS · 11234 NW 20th Street      QTS   
0  25.791496 -80.379582  QTS · 11234 NW 20th Street      QTS   
0  25.791496 -80.379582  QTS · 11234 NW 20th Street      QTS   
0  25.791496 -80.379582  QTS · 11234 NW 20th Street      QTS   
0  25.791496 -80.379582  QTS · 11234 NW 20th Street      QTS   
0  25.791496 -80.379582  QTS · 11234 NW 20th Street      QTS   

                           full_address  data_center_count  aggregated_power  \
0  11234 NW 20th Street, Miami, FL, USA                  1               2.0   
0  11234 NW 20th Street, Miami, FL, USA                  1               2.0   
0  1123

In [24]:
list(expanded_datacenter_df.columns)

['latitude',
 'longitude',
 'name',
 'provider',
 'full_address',
 'data_center_count',
 'aggregated_power',
 'aggregated_area',
 'yearbuilt',
 'sources',
 'avg_name_similarity',
 'avg_provider_similarity',
 'state',
 'facility_type_orig',
 'log_power',
 'log_area',
 'ftype_Building',
 'ftype_Campus',
 'ftype_Individual',
 'aggregated_power_imputed',
 'build_decade',
 'cluster',
 'cluster_mid_lat',
 'cluster_mid_lon',
 'year',
 'operational']

In [ ]:
# Merge the expanded datacenter data with the aggregated weather data on 'cluster' and 'year'
merged_df = pd.merge(
    expanded_datacenter_df, master_weather_df, on=["cluster", "year"], how="left"
)

# Display the first few rows to verify the merge
print("Merged DataFrame Sample:")
print(merged_df.head())

Merged DataFrame Sample:
   latitude_x  longitude_x                        name provider  \
0   25.791496   -80.379582  QTS · 11234 NW 20th Street      QTS   
1   25.791496   -80.379582  QTS · 11234 NW 20th Street      QTS   
2   25.791496   -80.379582  QTS · 11234 NW 20th Street      QTS   
3   25.791496   -80.379582  QTS · 11234 NW 20th Street      QTS   
4   25.791496   -80.379582  QTS · 11234 NW 20th Street      QTS   

                           full_address  data_center_count  aggregated_power  \
0  11234 NW 20th Street, Miami, FL, USA                  1               2.0   
1  11234 NW 20th Street, Miami, FL, USA                  1               2.0   
2  11234 NW 20th Street, Miami, FL, USA                  1               2.0   
3  11234 NW 20th Street, Miami, FL, USA                  1               2.0   
4  11234 NW 20th Street, Miami, FL, USA                  1               2.0   

   aggregated_area  yearbuilt                          sources  ...  \
0          38000.0  

In [30]:
list(merged_df.columns)

['latitude_x',
 'longitude_x',
 'name',
 'provider',
 'full_address',
 'data_center_count',
 'aggregated_power',
 'aggregated_area',
 'yearbuilt',
 'sources',
 'avg_name_similarity',
 'avg_provider_similarity',
 'state',
 'facility_type_orig',
 'log_power',
 'log_area',
 'ftype_Building',
 'ftype_Campus',
 'ftype_Individual',
 'aggregated_power_imputed',
 'build_decade',
 'cluster',
 'cluster_mid_lat',
 'cluster_mid_lon',
 'year',
 'operational',
 'time',
 'temperature_2m (°C)',
 'dew_point_2m (°C)',
 'relative_humidity_2m (%)',
 'wind_speed_10m (km/h)',
 'soil_temperature_0_to_7cm (°C)',
 'precipitation (mm)',
 'latitude_y',
 'longitude_y',
 'elevation']

In [31]:
merged_df = merged_df.drop(columns=['latitude_y', 'longitude_y'])

In [33]:
import pandas as pd

# Print current data types
print("Original Data Types:")
print(merged_df.dtypes)

# List of columns that should be numeric
cols_to_numeric = [
    'latitude_x', 'longitude_x', 'data_center_count', 'aggregated_power', 
    'aggregated_area', 'yearbuilt', 'log_power', 'log_area', 
    'aggregated_power_imputed', 'build_decade', 'cluster', 
    'cluster_mid_lat', 'cluster_mid_lon', 'year', 'operational',
    'temperature_2m (°C)', 'dew_point_2m (°C)', 'relative_humidity_2m (%)', 
    'wind_speed_10m (km/h)', 'soil_temperature_0_to_7cm (°C)', 'precipitation (mm)',
    'elevation'
]

# Convert specified columns to numeric; non-convertible values become NaN
for col in cols_to_numeric:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

# Convert 'time' column to datetime format
merged_df['time'] = pd.to_datetime(merged_df['time'], errors='coerce')

# Print the updated data types for verification
print("\nUpdated Data Types:")
print(merged_df.dtypes)

Original Data Types:
latitude_x                               float64
longitude_x                              float64
name                                      object
provider                                  object
full_address                              object
data_center_count                          int64
aggregated_power                         float64
aggregated_area                          float64
yearbuilt                                float64
sources                                   object
avg_name_similarity                      float64
avg_provider_similarity                  float64
state                                     object
facility_type_orig                        object
log_power                                float64
log_area                                 float64
ftype_Building                              bool
ftype_Campus                                bool
ftype_Individual                            bool
aggregated_power_imputed                 float64

In [35]:
import statsmodels.formula.api as smf
import numpy as np

# Optional: Create log-transformed control variables if not already done
# (If already available as 'log_power' and 'log_area', you can skip this)
merged_df['log_power'] = np.log1p(merged_df['aggregated_power'])
merged_df['log_area'] = np.log1p(merged_df['aggregated_area'])

# DID regression formula:
# - operational: 1 if the datacenter is active in that year (year >= yearbuilt)
# - year: controls for overall time trends
# - operational:year: interaction to capture the change in trend post-operation
# - log_power and log_area: controls for facility characteristics
# - C(cluster): cluster fixed effects to control for time-invariant cluster differences
formula = "Q('temperature_2m (°C)') ~ operational + year + operational:year + log_power + log_area + C(cluster)"

# Run the DID regression using OLS.
did_model = smf.ols(formula, data=merged_df.dropna(subset=['temperature_2m (°C)', 'operational', 'year', 'cluster'])).fit()

# Display the regression results
print("DID Regression Results for Temperature:")
print(did_model.summary())

DID Regression Results for Temperature:
                               OLS Regression Results                               
Dep. Variable:     Q('temperature_2m (°C)')   R-squared:                       0.817
Model:                                  OLS   Adj. R-squared:                  0.814
Method:                       Least Squares   F-statistic:                     261.6
Date:                      Sun, 23 Mar 2025   Prob (F-statistic):               0.00
Time:                              16:57:29   Log-Likelihood:                -7153.0
No. Observations:                      3456   AIC:                         1.442e+04
Df Residuals:                          3397   BIC:                         1.479e+04
Df Model:                                58                                         
Covariance Type:                  nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-------------------------

In [36]:
import pandas as pd

# Define the weather variables of interest
weather_vars = ['temperature_2m (°C)', 'dew_point_2m (°C)', 'relative_humidity_2m (%)',
                'wind_speed_10m (km/h)', 'soil_temperature_0_to_7cm (°C)', 'precipitation (mm)']

# For each datacenter (or cluster), compute average weather outcomes for pre-operational (operational==0) and post-operational (operational==1) years.
# Here, I group by datacenter name and cluster; if you prefer grouping by cluster only, adjust the groupby accordingly.

# Pre-operational averages
pre_df = (
    merged_df[merged_df['operational'] == 0]
    .groupby(['name', 'cluster'])[weather_vars]
    .mean()
    .reset_index()
    .rename(columns={var: var + '_pre' for var in weather_vars})
)

# Post-operational averages
post_df = (
    merged_df[merged_df['operational'] == 1]
    .groupby(['name', 'cluster'])[weather_vars]
    .mean()
    .reset_index()
    .rename(columns={var: var + '_post' for var in weather_vars})
)

# Merge the pre and post dataframes on datacenter identifier and cluster
diff_df = pd.merge(pre_df, post_df, on=['name', 'cluster'], how='inner')

# Calculate differences: (post - pre) for each weather variable
for var in weather_vars:
    diff_df[var + '_diff'] = diff_df[var + '_post'] - diff_df[var + '_pre']

# Let's inspect the differences for temperature as an example.
# For instance, if you consider an increase in temperature as a "positive effect" for demonstration,
# filter for rows where the difference in temperature is greater than 0.
temp_positive = diff_df[diff_df['temperature_2m (°C)_diff'] > 0]

print("Datacenters showing a positive difference in temperature (post - pre):")
print(temp_positive[['name', 'cluster', 'temperature_2m (°C)_pre', 'temperature_2m (°C)_post', 'temperature_2m (°C)_diff']])

# Similarly, you can examine differences for other weather variables:
print("\nDatacenters showing a positive difference in dew point (post - pre):")
print(diff_df[diff_df['dew_point_2m (°C)_diff'] > 0][['name', 'cluster', 'dew_point_2m (°C)_pre', 'dew_point_2m (°C)_post', 'dew_point_2m (°C)_diff']])

Datacenters showing a positive difference in temperature (post - pre):
                                           name  cluster  \
4                     3011 Lafayette St (SJC11)       23   
6             43881 Devin Shafron Drive (IAD12)       31   
18     Atlantic.Net · 2820 Northwestern Parkway       22   
22                          Centersquare · SFO4       23   
32                                Cologix · SV1       21   
33                               Colovore SJC01       23   
41                               CoreSite · SV3       22   
42                               CoreSite · SV4       22   
43                               CoreSite · SV5       22   
44                               CoreSite · SV7       22   
67       Digital Realty · 1550 Space Park Drive       23   
68       Digital Realty · 2045 Lafayette Street       20   
69       Digital Realty · 2055 Lafayette Street       20   
72                       Digital Realty · ACC 2       34   
73                       Digi

In [40]:
import pandas as pd

# List the weather variables of interest
weather_vars = [
    'temperature_2m (°C)',
    'dew_point_2m (°C)',
    'relative_humidity_2m (%)',
    'wind_speed_10m (km/h)',
    'soil_temperature_0_to_7cm (°C)',
    'precipitation (mm)'
]

# Define the expected effect direction:
# 1 means an increase is considered a positive effect,
# -1 means a decrease is considered a positive effect.
# Adjust these based on your hypothesis.
expected_effect = {
    'temperature_2m (°C)': 1,
    'dew_point_2m (°C)': 1,
    'relative_humidity_2m (%)': 1,    # or -1 if you expect it to decrease
    'wind_speed_10m (km/h)': -1,       # suppose you expect wind speed to drop due to obstructions
    'soil_temperature_0_to_7cm (°C)': 1,
    'precipitation (mm)': 1           # or -1, depending on your theory
}

# Compute average values for pre-operational (operational == 0) and post-operational (operational == 1) years by cluster
pre_df = (
    merged_df[merged_df['operational'] == 0]
    .groupby('cluster')[weather_vars]
    .mean()
    .reset_index()
    .rename(columns={var: var + '_pre' for var in weather_vars})
)

post_df = (
    merged_df[merged_df['operational'] == 1]
    .groupby('cluster')[weather_vars]
    .mean()
    .reset_index()
    .rename(columns={var: var + '_post' for var in weather_vars})
)

# Merge pre and post summaries on cluster
diff_df = pd.merge(pre_df, post_df, on='cluster', how='inner')

# Calculate the difference (post - pre) for each weather variable
for var in weather_vars:
    diff_df[var + '_diff'] = diff_df[var + '_post'] - diff_df[var + '_pre']

print("Differences by Cluster:")
print(diff_df[['cluster'] + [var + '_diff' for var in weather_vars]])

# Filter clusters based on expected effect for each variable
filtered_clusters = {}
for var in weather_vars:
    # If expected_effect is 1, we want diff > 0, if -1, diff < 0.
    sign = expected_effect[var]
    if sign == 1:
        clusters_pos = set(diff_df.loc[diff_df[var + '_diff'] > 0, 'cluster'])
    elif sign == -1:
        clusters_pos = set(diff_df.loc[diff_df[var + '_diff'] < 0, 'cluster'])
    else:
        clusters_pos = set()  # or handle 0 as a neutral case if needed
    filtered_clusters[var] = clusters_pos
    print(f"\nClusters with expected effect in {var}:")
    print(clusters_pos)

# If desired, find common clusters across all variables where each shows the expected direction
common_clusters = set.intersection(*filtered_clusters.values())
print("\nCommon clusters with expected effects across all selected variables:")
print(common_clusters)

Differences by Cluster:
    cluster  temperature_2m (°C)_diff  dew_point_2m (°C)_diff  \
0         0                 -0.025402                0.340822   
1         1                 -0.558215               -0.545290   
2         2                 -1.023032               -0.609418   
3         3                 -0.234932               -0.254363   
4         4                 -0.289085                0.149257   
..      ...                       ...                     ...   
57       57                 -1.299781               -1.296970   
58       58                 -1.448020               -1.121326   
59       59                 -0.903864               -0.761960   
60       60                 -0.040425               -0.138353   
61       61                 -0.095189               -0.957922   

    relative_humidity_2m (%)_diff  wind_speed_10m (km/h)_diff  \
0                        1.712734                    0.056411   
1                       -0.080730                   -0.219276   


In [41]:
# Intersection for temperature and dew point only:
common_temp_dew = pos_clusters['temperature_2m (°C)'].intersection(pos_clusters['dew_point_2m (°C)'])
print("Common clusters for temperature and dew point:", common_temp_dew)

Common clusters for temperature and dew point: {15}


In [47]:
import folium
from shapely.geometry import MultiPoint
from numpy import array
import random

# Create a base map centered on the US
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Generate a random color for each cluster
# Using a fixed seed for consistent colors
random.seed(42)
cluster_colors = {
    cluster_id: '#' + ''.join(random.choices('0123456789ABCDEF', k=6))
    for cluster_id in positive_temp_clusters
}

# For each cluster, create a polygon around its datacenters
for cluster_id in positive_temp_clusters:
    # Get the color for this cluster
    color = cluster_colors[cluster_id]
    
    # Get all datacenters in this cluster
    cluster_dcs = merged_df[merged_df['cluster'] == cluster_id][['name', 'latitude_x', 'longitude_x']].drop_duplicates()
    
    # Create circle markers for each datacenter with the cluster's color
    for _, dc in cluster_dcs.iterrows():
        folium.CircleMarker(
            location=[dc['latitude_x'], dc['longitude_x']],
            radius=5,
            popup=f"Name: {dc['name']}<br>Cluster: {cluster_id}",
            tooltip=dc['name'],
            color=color,
            fill=True,
            fillColor=color,
            fillOpacity=0.7
        ).add_to(m)
    
    # Create a polygon around the datacenters if there are at least 3 points
    points = cluster_dcs[['latitude_x', 'longitude_x']].values
    if len(points) >= 3:
        # Create a convex hull around the points
        hull = MultiPoint(array([[p[1], p[0]] for p in points])).convex_hull
        if hull.geom_type == 'Polygon':
            # Convert the polygon coordinates to the format expected by folium
            coords = [[y, x] for x, y in hull.exterior.coords]
            
            # Add the polygon to the map with the same color as the markers
            folium.Polygon(
                locations=coords,
                color=color,
                fill=True,
                fillColor=color,
                fillOpacity=0.1,
                weight=2,
                popup=f'Cluster {cluster_id}'
            ).add_to(m)

# Display the map
m